In [1]:
import pandas as pd
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer, create_optimizer, AdamWeightDecay, TFAutoModelForCausalLM




2022-10-20 20:21:55.299150: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/ArjunPatel/opt/anaconda3/envs/whisper-gpt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#most code is taken from the old huggingface script for language modeling with 
DATASET_PATH = "kpriyanshu256/whisper-transcripts"
MODEL_CHECKPOINT = "distilgpt2"
BLOCK_SIZE = 64
EPOCHS = 1


In [62]:

def tokenize_function(dat, model_checkpoint = MODEL_CHECKPOINT):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    return tokenizer(dat["text"])


def group_texts(dat, block_size = BLOCK_SIZE):
    # function from HF script used to chunk data into block_size
    # Concatenate all texts.
    concatenated_examples = {k: sum(dat[k], []) for k in dat.keys()}
    total_length = len(concatenated_examples[list(dat.keys())[0]])
    # We drop the small remainder, though you could add padding instead if the model supports it
    # In this, as in all things, we advise you to follow your heart
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + BLOCK_SIZE] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result



def compile_model(model_checkpoint = MODEL_CHECKPOINT, lr = 2e-5, weight_decay_rate = 0.01):
    # Retrieve a model from model_checkpoint, and load with optimizer
    model = TFAutoModelForCausalLM.from_pretrained(model_checkpoint)

    optimizer = AdamWeightDecay(lr=lr, weight_decay_rate=weight_decay_rate)

    model.compile(optimizer=optimizer)
    return model



In [63]:
# tokenize and preprocess dataset
datasets = load_dataset(DATASET_PATH)

Using custom data configuration kpriyanshu256--whisper-transcripts-b6043c49b20f7241
Found cached dataset json (/Users/ArjunPatel/.cache/huggingface/datasets/kpriyanshu256___json/kpriyanshu256--whisper-transcripts-b6043c49b20f7241/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 358.55it/s]


In [64]:
# apply tokenization
tokenized_datasets = datasets.map(
    tokenize_function, 
    batched=True, 
    num_proc=4, 
    remove_columns = ["text", "id", "segments"])


# chunk the data
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


model = compile_model(MODEL_CHECKPOINT)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]


#1: 100%|██████████| 1/1 [00:01<00:00,  1.02s/ba]
Token indices sequence length is longer than the specified maximum sequence length for this model (1886 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2072 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1255 > 1024). Running this sequence through the model will result in indexing errors


#0: 100%|██████████| 1/1 [00:01<00:00,  1.18s/ba]

#0: 100%|██████████| 1/1 [00:00<00:00, 17.30ba/s]


#3: 100%|██████████| 1/1 [00:00<00:00, 21.71ba/s]
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to th

In [65]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

train_set = lm_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

In [66]:
mod_history = model.fit(train_set, epochs=EPOCHS)

10/41 [======>.......................] - ETA: 1:17 - loss: 3.9420

KeyboardInterrupt: 

In [67]:
model.save("trained_model")

INFO:tensorflow:Assets written to: trained_model/assets


INFO:tensorflow:Assets written to: trained_model/assets
